Данное домашнее задание посвящено схеме базе данных, связанной с футбольным командами. Схема расположена в hw_diagram.jpg
## Описание таблиц 

### Positions 
Позиции игроков
id - уникальный идентификатор позиции 
Name - Название таблицы 

### Coaches 
Таблица тренеров 
id - уникальный идентификатор тренера
Name - имя тренера 

## Teams 
Таблица команд 
id - уникальный идентификатор  команды 
Name - имя команды 
Location - расположение команды 
Coach_id - идентификатор тренера 

### Players 
Таблица игроков 
Id - уникальный идентификатор игрока 
Name - имя игрока 
Age - возраст игрока 
Position_id - идентификатор позиции 
Team_id - идентификатор команды 
Start_date - дата начала игры в команде 

### PlayersArchive 
Архив по игрокам (информация о том, в какой команде игрок играл ранее) 
player_id - идентификатор игрока 
team_id - идентификатор команды 
start_date - дата начала игры 
end_date - дата окончания игры

In [1]:
%load_ext sql
%sql sqlite:///football.db

u'Connected: None@football.db'

# Задание 1
Реализовать перенакатываемые sql-скрипты по схеме hw_diagram, приложенного ниже.

In [126]:
%%sql
pragma foreign_keys=oт;
drop table if exists coaches;
create table coaches(
    id int primary key,
    name varchar(10)
);

drop table if exists positions;
create table positions(
    id int primary key,
    name varchar(10)
);

drop table if exists teams;
create table teams(
    id int primary key,
    name varchar(10),
    location varchar(10),
    coach_id int,
    foreign key (id) references playersarchive(team_id),
    foreign key (coach_id) references coaches(id)
);



drop table if exists playersarchive;
create table playersarchive(
    player_id int,
    team_id int,
    start_date date,
    end_date date,
    primary key(player_id,team_id),
    foreign key (team_id) references teams(id)
);

drop table if exists players;
create table players(
    id int primary key,
    name varchar(10),
    age varchar(10),
    position_id int,
    team_id int,
    start_date date,
    foreign key (id) references playersarchive(player_id),
    foreign key (position_id) references positions(id),
    foreign key (team_id) references teams(id)
);

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

# Задание 2
Написать sql-скрипт, осуществляюший добавление всех возможных позиций игроков. (В данной задаче можно ограничится четырьмя - вратарь, защитник, полузащитник, нападающий)

In [127]:
%%sql
insert into positions values('1','goalkeeper');
insert into positions values('2','defender');
insert into positions values('3','midfielder');
insert into positions values('4','forward');

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [128]:
%%sql
select *
from positions

Done.


id,name
1,goalkeeper
2,defender
3,midfielder
4,forward


# Задание 3
Написать функцию, осуществляющую добавление тренера. На вход функции подается имя тренера. При невозможности вставки функция должна обработать данное исключение и вывести пользователю информацию о проблеме

После написания функции необходимо добавить несколько тренеров для таблицы.

In [129]:
import sqlite3
db=sqlite3.connect('football.db')

def coaches(args):
    cursor=db.cursor()
    cursor.execute('''select max(id) from coaches''')
    a=cursor.fetchone()[0]
    cursor.execute('''select id from coaches where name=?''',(args,))
    try:
        b=cursor.fetchone()[0]
    except:
        print()
    else:
        print('already exists')
        return -1
    try:
        a+=1
    except:
        a=1   
    try:
        cursor.execute('''insert into coaches values(?,?)''',(a,args))
        db.commit()
    except:
        print('problem')
    

In [130]:
coaches('Vista')
coaches('Alexander')
coaches('Olga')

()
()
()


In [131]:
%sql select * from coaches

Done.


id,name
1,Vista
2,Alexander
3,Olga


# Задание 4

Написать функцию, осуществляющую добавление команды. На вход функция принимает (имя команды, местоположение, имя тренера).
На выходе должна быть добавлена команда.

После написания функции необходимо добавить несколько команд

In [132]:
def team(name,location,coach):
    cursor=db.cursor()
    cursor.execute('''select max(id) from teams''')
    a=cursor.fetchone()[0]
    cursor.execute('''select id from coaches where name=?''',(coach,))
    coach=cursor.fetchone()[0]
    try:
        a+=1
    except:
        a=1
    cursor.execute('''insert into teams values(?,?,?,?)''', (a,name,location,coach)) 
    db.commit()

In [133]:
team('spartak', 'moscow','Vista')
team('dinamo','moscow','Olga')
team('zenit','sp','Alexander')

In [134]:
%sql select * from teams

Done.


id,name,location,coach_id
1,spartak,moscow,1
2,dinamo,moscow,3
3,zenit,sp,2


# Задание 5

Написать функцию, осуществляющую обновление информации о команде. У функции на входе должен быть один обязательный параметр - название команды и два необязательных параметра - имя тренера и местоположение. При этом хотя бы один из необязательных параметров должен быть заполнен. Функция должна обновлять значение переданных необязательных параметров.

После написания функции вызовите данную функцию несколько раз

In [135]:
def upd(args):
    cursor=db.cursor()
    cursor.execute('''select id from coaches where name=?''',(args[1],))
    flag=0
    try:
        a=cursor.fetchone()[0]
    except:
        flag=1
    else:
        flag=2
        args[1]=a
    if flag==2:
        try:
            print(args[flag])
        except:
            flag=3
        else:
            flag=4
    print(flag)
    if flag==4:
        cursor.execute('''update teams set coach_id=?, location=? where name=?''',(args[1],args[2],args[0],))
    elif flag==3:
        cursor.execute('''update teams set coach_id=? where name=?''',(args[1],args[0],))
    else:
        cursor.execute('''update teams set location=? where name=?''', (args[1],args[0],))
    db.commit()

In [136]:
%sql select * from teams

Done.


id,name,location,coach_id
1,spartak,moscow,1
2,dinamo,moscow,3
3,zenit,sp,2


In [137]:
upd(['dinamo','Vista','moscow'])

moscow
4


# Задание 6

Написать функцию, осуществляющую добавление нового футболиста в таблицу игроков. На входе (имя, возраст, название позиции, название команды, дата перехода в команду(необязательный параметр)). Если дата перехода не указана, то нужно использовать текущую дату. Предусмотреть различные ошибки (такие как неверное имя позиции, неверное название команды, возраст < 10 и > 50 и т.п.)

После написания функции необходимо добавить несколько игроков.

In [138]:
import datetime

def player(*args):
    cursor=db.cursor()
    name=args[0]
    if args[1]<10 and args[1]>50:
        return -1
    age=args[1]
    cursor.execute('''select id from positions where name=?''',(args[2],))
    try:
        position=cursor.fetchone()[0]
    except:
        return -2
    cursor.execute('''select id from teams where name=?''',(args[3],))
    try:
        team=cursor.fetchone()[0]
    except:
        return -3
    length=len(args)
    if length==4:
        date=datetime.date.today()
    else:
        date=args[4]
    cursor.execute('''select max(id) from players''')
    maxid=cursor.fetchone()[0]
    try:
        maxid+=1
    except:
        maxid=1
    cursor.execute('''insert into players values(?,?,?,?,?,?)''',(maxid,name,age,position,team,date))
    db.commit()

In [139]:
player('Martin','20','goalkeeper','spartak')

In [140]:
%sql select * from players


Done.


id,name,age,position_id,team_id,start_date
1,Martin,20,1,1,2017-05-06


# Задание 7
Написать функцию для перевода игрока в другую команду. Данная функция должна осуществить архивирование текущей записи в таблице игроков в таблицу players_archive с заполнение end_date - датой начала игры за новую команду - 1 день. А также осуществлять изменение поля в таблице Players. При возникновении ошибки должен осуществляться полный откат транзакции.

In [145]:
def updp(name, team,datet):
    cursor=db.cursor()
    cursor.execute('''select id from players where name=?''',(name))
    try:
        playerid=cursor.fetchone()[0]
    except:
        flag=1
    cursor.execute('''select end_date from playersarchive where id=?''',(playerid))
    try:
        date=cursor.fetchone()[0]
    except:
        flag=1
    cursor.execute('''select id from teams where name=?''',(team))
    try:
        team=cursor.fetchone()[0]
    except:
        flag=1
    if flag:
        db.rollback() 
        return -1
    try:
        cursor.execute('''update playersarchive set start_date=?, team_id=?,end_date=? where player_id=?''',(date,team,datet-1,playerid))
        cursor.execute('''update players set team_id=?,start_date=? where id=?''',(team,datet,playerid))
    except:
        db.rollback()
        return -1
        
    db.commit()

In [146]:
%%sql 
insert into playersarchive values(1,1,'2010-10-10','2011-11-11')

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: playersarchive.player_id, playersarchive.team_id [SQL: u"insert into playersarchive values(1,1,'2010-10-10','2011-11-11')"]

In [143]:
%sql select * from playersarchive

Done.


player_id,team_id,start_date,end_date
1,1,2010-10-10,2011-11-11
